In [2]:
import pickle
import glob
import os
import pandas as pd
import numpy as np
import heapq

In [3]:
test_news_path = '../infer/mind/large_test/news.tsv'
bias_detection_path = 'data/0929_concat_mind_over_res.csv'
infer_dir = '../infer/interest_infer_lstur/'
infer_dir_processed = '../infer/interest_infer_lstur/processed'
candidate_news_file = 'data/news_new_with_time.tsv'

In [5]:
!ls -ahl $infer_dir

total 467632
drwxr-xr-x  14 qin  staff   448B  8 May 02:00 .
drwxr-xr-x  16 qin  staff   512B  1 Jun 09:49 ..
-rw-r--r--   1 qin  staff   194M  8 May 00:59 interest_behaviors_high_0_news.pkl
-rw-r--r--   1 qin  staff   3.2M  8 May 01:00 interest_behaviors_high_0_user.pkl
-rw-r--r--   1 qin  staff   3.2M  8 May 01:22 interest_behaviors_high_1_user.pkl
-rw-r--r--   1 qin  staff   3.2M  8 May 01:24 interest_behaviors_high_2_user.pkl
-rw-r--r--   1 qin  staff   3.2M  8 May 01:27 interest_behaviors_high_3_user.pkl
-rw-r--r--   1 qin  staff   3.2M  8 May 01:28 interest_behaviors_high_4_user.pkl
-rw-r--r--   1 qin  staff   3.2M  8 May 01:29 interest_behaviors_low_0_user.pkl
-rw-r--r--   1 qin  staff   3.2M  8 May 01:31 interest_behaviors_low_1_user.pkl
-rw-r--r--   1 qin  staff   3.2M  8 May 01:32 interest_behaviors_low_2_user.pkl
-rw-r--r--   1 qin  staff   3.2M  8 May 01:33 interest_behaviors_low_3_user.pkl
-rw-r--r--   1 qin  staff   3.2M  8 May 01:34 interest_behaviors_low_4_user.pkl
drwx

### convert user dict to user vecs

In [6]:
os.makedirs(infer_dir_processed, exist_ok=True)

In [7]:
# convert user dict to user vecs

for file_name in sorted(glob.glob(os.path.join(infer_dir, '*user.pkl'))):
    with open(file_name, 'rb') as f:
        user_dict = pickle.load(f)
        
    v_list = []
    for k, v in user_dict.items():
        v_list.append(v)

    v_vec = np.asarray(v_list)
    file_suffix = file_name.split("/")[-1].split(".pkl")[0]

    with open(os.path.join(infer_dir_processed, f'{file_suffix}.npy'), 'wb') as f:
        pickle.dump(v_vec, f)

In [8]:
!ls $infer_dir_processed

candiate_news_vec.npy              interest_behaviors_high_4_user.npy
candiate_nid_list.pkl              interest_behaviors_low_0_user.npy
interest_behaviors_high_0_user.npy interest_behaviors_low_1_user.npy
interest_behaviors_high_1_user.npy interest_behaviors_low_2_user.npy
interest_behaviors_high_2_user.npy interest_behaviors_low_3_user.npy
interest_behaviors_high_3_user.npy interest_behaviors_low_4_user.npy


### find vectors of candidate news

In [9]:
!wc -l $test_news_path

  120961 /Users/qin/2022/NewsRec/data/mind/large_test/news.tsv


In [10]:
nid_list = []
with open(test_news_path, 'r') as f:
    for line in f:
        line_split = line.split('\t')
        nid_list.append(line_split[0])
print(len(nid_list))

120961


In [11]:
with open(os.path.join(infer_dir, 'interest_behaviors_high_0_news.pkl'), 'rb') as f:
    full_news_dict = pickle.load(f)

In [12]:
len(nid_list), len(full_news_dict)

(120961, 120962)

In [13]:
nid2vecs = dict()
for idx, nid in enumerate(nid_list):
    nid2vecs[nid] = full_news_dict[idx+1]

In [14]:
news_df = pd.read_csv(candidate_news_file, sep='\t', header=None)
news_df.columns = ['nid', 'category', 'subcategory', 'title', 'abstract', 'url', 'entity1', 'entity2', 'timestamp']
news_df.shape

(22283, 9)

In [15]:
nid_list = news_df['nid'].tolist()
len(nid_list)

22283

In [16]:
nid_vecs_arr = []
candidate_nid = []
for nid in nid_list:
    nid_vecs_arr.append(nid2vecs[nid])
    candidate_nid.append(nid)

In [17]:
with open(os.path.join(infer_dir_processed, f'candiate_nid_list.pkl'), 'wb') as f:
     pickle.dump(candidate_nid, f)

In [18]:
nid_vecs_arr = np.asarray(nid_vecs_arr)

In [19]:
nid_vecs_arr.shape

(22283, 400)

In [20]:
with open(os.path.join(infer_dir_processed, f'candiate_news_vec.npy'), 'wb') as f:
     pickle.dump(nid_vecs_arr, f)

### results

In [23]:

# for file_name in sorted(glob.glob(os.path.join(infer_dir_processed, '*user.npy'))):
#     file_suffix = file_name.split("/")[-1].split(".npy")[0]
#     with open(os.path.join(infer_dir_processed, f'candiate_news_vec.npy'), 'rb') as f:
#          news_vec = pickle.load(f)
#     with open(file_name, 'rb') as f:
#          user_vec = pickle.load(f)
#     with open(os.path.join(infer_dir_processed, f'candiate_nid_list.pkl'), 'rb') as f:
#          candidate_nid = pickle.load(f)
            
#     bias_detection_df = pd.read_csv(bias_detection_path, header=None, sep='\t')
#     bias_detection_df.columns = ['nid', 'category', 'sub_category', 'bias_prob']

#     nis2bias_prob = dict(zip(bias_detection_df['nid'], bias_detection_df['bias_prob']))

#     a = np.dot(user_vec, news_vec.T)

#     top_n = 100
#     aa_arr = []
#     for aa in a:
#         ind = heapq.nlargest(top_n, range(len(aa)), aa.take)
#         aa_prob_list = [nis2bias_prob[candidate_nid[idx]] for idx in ind]
#         aa_arr.append(aa_prob_list)
#     aa_arr = np.asarray(aa_arr)

#     look_num_list = [i*10 for i in range(1, 11)]
#     mean_val_list = []
#     std_val_list = []
#     display_score_list = []
#     for look_num in look_num_list:
#         bb_arr = aa_arr[:, :look_num]
#         bias_prob_list = []
#         for bb in bb_arr:
#             temp = np.count_nonzero(bb >= 0.5)
#             bias_prob_list.append(temp/len(bb))
#         bias_prob_list = np.asarray(bias_prob_list)
        
#         mean_val = np.mean(bias_prob_list, dtype=np.float64)
#         std_val = np.std(bias_prob_list, dtype=np.float64)
#         display_score = f'{mean_val*100:.2f}'
#         mean_val_list.append(mean_val)
#         std_val_list.append(std_val)
#         display_score_list.append(display_score)
    
#     file_suffix = file_suffix.split("_user")[0]
#     print(file_suffix)
#     print('mean', mean_val_list)
#     print('std', std_val_list)
#     print('display', display_score_list)
#     print('*'*10)

interest_behaviors_high_0
mean [0.14935, 0.162375, 0.1644666666666667, 0.1638875, 0.16321000000000005, 0.16345833333333334, 0.16295714285714286, 0.16276875000000002, 0.16189444444444445, 0.160995]
std [0.14835625197476515, 0.13584222235740992, 0.12737365853617022, 0.12002062257691384, 0.11510297954440624, 0.11207939119461506, 0.10960350713143355, 0.10803792574109103, 0.1064766999087443, 0.10507835160012743]
display ['14.94', '16.24', '16.45', '16.39', '16.32', '16.35', '16.30', '16.28', '16.19', '16.10']
**********
interest_behaviors_high_1
mean [0.10160000000000001, 0.10085000000000002, 0.09841666666666667, 0.0979375, 0.09696, 0.09586666666666667, 0.09506428571428571, 0.09445625, 0.09348888888888889, 0.092905]
std [0.114531393076309, 0.0926648665892311, 0.08332829151414702, 0.07939495320075451, 0.07683722014753007, 0.07429202140502332, 0.0724288951105757, 0.07119713801086038, 0.06939385179298428, 0.06880051580475251]
display ['10.16', '10.09', '9.84', '9.79', '9.70', '9.59', '9.51', '